In [8]:
import os
import glob
import pandas as pd
def get_data(dn):
    data = {
        "news":[],
        "ans":[]
    }
    dirs = glob.glob(os.path.join(dn, "*"))
    for d in dirs:
        pat = os.path.join(d, "*.[tT][xX][tT]")
        fs = glob.glob(pat)
        for fp in fs:
            with open(fp, "r", encoding="utf-8") as f:
                news = f.read()
            ans = os.path.split(d)[-1]
            data["news"].append(news)
            data["ans"].append(ans)
    df = pd.DataFrame(data, columns=["news", "ans"])
    return df

In [10]:
train_df = get_data("chinese_news_trans/chinese_news_trans")
test_df = get_data("chinese_news_test/chinese_news_test")
test_df

,news,ans
0,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
1,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
2,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
3,三趟火車停開 乘客可全額退票 \n瀏覽次數：1180 \n 昨日，來自鐵路部門的消息說...,交通
4,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
...,...,...
96,最優秀選手無緣亞運會健美賽\n \n \n 健美在亞洲運動會上是“新生兒”——韓國釜...,體育
97,\n各國記者眼中的羽毛球世錦賽\n \n\n-----------------------...,體育
98,\n友好運動會第五天 東道主選手大顯神威\n2001年09月03日02:51:40 新華社 ...,體育
99,不靠技術比運氣\n\n 第二屆奧運會在巴黎舉行，同時這裡也正在舉行國際博覽會，東道主把一些...,體育


In [19]:
# unique value_counts
kind = train_df["ans"].unique()
# enumerate
trans = {k:i for i, k in enumerate(kind)}
trans

{'交通': 0,
 '政治': 1,
 '教育': 2,
 '環境': 3,
 '經濟': 4,
 '藝術': 5,
 '計算機': 6,
 '軍事': 7,
 '醫藥': 8,
 '體育': 9}